In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier as dt


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train = pd.read_csv('datathon2020_train.csv')
train

,ID,F2,F3,F4,F8,F11,F25,F39,F52,F53,...,F127,F128,F129,F130,F131,F132,F133,F136,F139,COVID_TCPM
0,10681,1,1,1,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate with dry winter, and Warm summer",9.028250e+04,0.367070,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.102565,low
1,3300,1,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer",2.885228e+05,0.420663,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.186848,high
2,12038,1,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer",9.469753e+04,0.343825,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.131743,low
3,10236,0,1,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical monsoon,1.753151e+05,0.558702,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,preliminary,0.703491,low
4,4111,1,1,1,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Warm summer",8.909464e+04,0.508010,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.531441,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,5740,0,1,1,Deserts and Xeric Shrublands,"Desert (arid), and Hot arid",2.051100e+05,0.094838,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.000000,high
496,554,1,1,1,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Hot summer",1.733145e+06,0.515650,2.631636e+06,2.385705e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,available,0.731324,high
497,12566,1,1,1,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate, fully humid, and Hot summer",3.038501e+06,0.315763,1.613929e+05,3.182036e+05,...,0.946529,1.200975,1.281726,7113.034668,8993.411865,10894.03711,12675.01086,available,0.034969,low
498,13005,0,1,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest,7.359616e+05,0.439870,3.912408e+01,6.799629e+02,...,16.289423,19.483387,20.412186,88959.949020,281104.611600,456585.33630,735961.58750,preliminary,0.342686,low


In [3]:
test = pd.read_csv('datathon2020_test.csv')
test

,ID,F2,F3,F4,F8,F11,F25,F39,F52,F53,...,F127,F128,F129,F130,F131,F132,F133,F136,F139,COVID_TCPM
0,12272,1,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer",3.010114e+05,0.270918,NaN,526075.1429,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.008360,low
1,6526,1,1,1,Deserts and Xeric Shrublands,"Desert (arid), and Hot arid",8.668036e+04,0.303313,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.000000,low
2,11320,2,1,1,Temperate Broadleaf and Mixed Forests,"Mild temperate with dry winter, and Hot summer",2.972672e+05,0.359829,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.165453,low
3,4566,0,0,1,Temperate Broadleaf and Mixed Forests,"Mild temperate with dry summer, and Hot summer",9.294967e+04,0.274426,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.029401,low
4,4866,1,1,1,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate, fully humid, and Warm summer",1.546434e+05,0.475473,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.334338,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,1575,1,1,1,"Mediterranean Forests, Woodlands, and Scrub","Mild temperate with dry summer, and Hot summer",3.985660e+06,0.262157,168320.0651,367945.1962,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.040463,low
2482,13009,0,0,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest,7.704883e+04,0.388311,NaN,NaN,...,2.593406,2.593406,2.688027,13306.76338,37205.58549,55806.48722,77048.82957,preliminary,0.221624,low
2483,11032,0,1,1,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest,5.288064e+04,0.556476,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.794566,low
2484,7213,0,1,1,Tropical and Subtropical Dry Broadleaf Forests,Tropical savannah with dry winter,1.618023e+05,0.405874,NaN,NaN,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,available,0.107804,low


In [4]:
train.isna().sum()

ID              0
F2              0
F3              0
F4              0
F8              0
F11             0
F25             0
F39            14
F52           364
F53           350
F54           349
F55           328
F69            23
F72           453
F73           437
F74           418
F75           389
F76             3
F77             3
F78             1
F79             3
F81           271
F82           181
F83           165
F84           463
F85           462
F86           394
F92           361
F93           349
F94           348
F95           328
F98             0
F99             0
F109            0
F110            0
F116            0
F121            0
F122            0
F123            0
F124            0
F125            0
F126            0
F127            0
F128            0
F129            0
F130            0
F131            0
F132            0
F133            0
F136            0
F139            0
COVID_TCPM      0
dtype: int64

In [5]:
test.isna().sum()

ID               0
F2               0
F3               0
F4               0
F8               3
F11              8
F25              0
F39             91
F52           1789
F53           1690
F54           1690
F55           1629
F69            113
F72           2286
F73           2179
F74           2102
F75           1925
F76             15
F77             15
F78             10
F79             11
F81           1314
F82            967
F83            908
F84           2293
F85           2286
F86           1970
F92           1780
F93           1685
F94           1685
F95           1625
F98              0
F99              0
F109             1
F110             1
F116             0
F121             0
F122             0
F123             0
F124             0
F125             0
F126             0
F127             0
F128             0
F129             0
F130             0
F131             0
F132             0
F133             0
F136             0
F139             0
COVID_TCPM       0
dtype: int64

In [6]:
test.dtypes

ID              int64
F2              int64
F3              int64
F4              int64
F8             object
F11            object
F25           float64
F39           float64
F52           float64
F53           float64
F54           float64
F55           float64
F69           float64
F72           float64
F73           float64
F74           float64
F75           float64
F76           float64
F77           float64
F78           float64
F79           float64
F81           float64
F82           float64
F83           float64
F84           float64
F85           float64
F86           float64
F92           float64
F93           float64
F94           float64
F95           float64
F98           float64
F99           float64
F109          float64
F110          float64
F116            int64
F121          float64
F122          float64
F123          float64
F124          float64
F125            int64
F126          float64
F127          float64
F128          float64
F129          float64
F130      

In [7]:
train[['F8', 'F11']]

,F8,F11
0,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate with dry winter, and Warm summer"
1,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer"
2,Temperate Broadleaf and Mixed Forests,"Mild temperate, fully humid, and Hot summer"
3,Tropical and Subtropical Moist Broadleaf Forests,Tropical monsoon
4,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Warm summer"
...,...,...
495,Deserts and Xeric Shrublands,"Desert (arid), and Hot arid"
496,Temperate Broadleaf and Mixed Forests,"Snow, fully humid, and Hot summer"
497,Tropical and Subtropical Moist Broadleaf Forests,"Mild temperate, fully humid, and Hot summer"
498,Tropical and Subtropical Moist Broadleaf Forests,Tropical rain forest


F8 - describes the land conditions of the specific region

F11 - describes the weather conditions of the specific region

In [8]:
na_percents = train.isna().sum()/len(train)*100
na_percents

ID             0.0
F2             0.0
F3             0.0
F4             0.0
F8             0.0
F11            0.0
F25            0.0
F39            2.8
F52           72.8
F53           70.0
F54           69.8
F55           65.6
F69            4.6
F72           90.6
F73           87.4
F74           83.6
F75           77.8
F76            0.6
F77            0.6
F78            0.2
F79            0.6
F81           54.2
F82           36.2
F83           33.0
F84           92.6
F85           92.4
F86           78.8
F92           72.2
F93           69.8
F94           69.6
F95           65.6
F98            0.0
F99            0.0
F109           0.0
F110           0.0
F116           0.0
F121           0.0
F122           0.0
F123           0.0
F124           0.0
F125           0.0
F126           0.0
F127           0.0
F128           0.0
F129           0.0
F130           0.0
F131           0.0
F132           0.0
F133           0.0
F136           0.0
F139           0.0
COVID_TCPM     0.0
dtype: float

In [9]:
cols_to_drop = na_percents[na_percents > 50.0].index.tolist()
cols_to_drop
train_copy_drops = train.drop(cols_to_drop, axis=1)
train_copy_drops.isna().sum() / len(train_copy_drops) * 100

['F52',
 'F53',
 'F54',
 'F55',
 'F72',
 'F73',
 'F74',
 'F75',
 'F81',
 'F84',
 'F85',
 'F86',
 'F92',
 'F93',
 'F94',
 'F95']

ID             0.0
F2             0.0
F3             0.0
F4             0.0
F8             0.0
F11            0.0
F25            0.0
F39            2.8
F69            4.6
F76            0.6
F77            0.6
F78            0.2
F79            0.6
F82           36.2
F83           33.0
F98            0.0
F99            0.0
F109           0.0
F110           0.0
F116           0.0
F121           0.0
F122           0.0
F123           0.0
F124           0.0
F125           0.0
F126           0.0
F127           0.0
F128           0.0
F129           0.0
F130           0.0
F131           0.0
F132           0.0
F133           0.0
F136           0.0
F139           0.0
COVID_TCPM     0.0
dtype: float64

In [40]:
test_copy_drops = test.drop(cols_to_drop, axis=1)
F8_F11_df = test_copy_drops[['F8', 'F11', 'F136']]
rest_test = test_copy_drops.drop(['F8', 'F11', 'F136', 'COVID_TCPM'], axis=1)
#test_copy_drops.isna().sum()

In [41]:
imp1 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp1 = imp1.fit(F8_F11_df)
imputed_F8_F11_df_test = pd.DataFrame(imp1.transform(F8_F11_df), columns=F8_F11_df.columns)
imputed_F8_F11_df_test.isna().sum()

F8      0
F11     0
F136    0
dtype: int64

In [43]:
imp2 = SimpleImputer(missing_values=np.nan, strategy='mean')
imp2 = imp2.fit(rest_test)
imputed_rest_df = pd.DataFrame(imp2.transform(rest_test), columns=rest_test.columns)

In [50]:
no_nan_test = pd.concat([imputed_rest_df,imputed_F8_F11_df_test], axis=1)
no_nan_test.to_csv('clean_test.csv', index=False)

In [53]:
no_nan_test['F8'] = no_nan_test['F8'].astype('category')
no_nan_test['F11'] = no_nan_test['F11'].astype('category')
no_nan_test['F136'] = no_nan_test['F136'].astype('category')
df_test_onehot = pd.get_dummies(no_nan_test, columns=['F8', 'F11', 'F136'], prefix=['F8_Area_Condition', 'F11_Climate', 'F136'])
df_test_onehot.head(5)
df_test_onehot.shape

,ID,F2,F3,F4,F25,F39,F69,F76,F77,F78,...,F8_Area_Condition_Tropical and Subtropical Moist Broadleaf Forests;Mangroves,F8_Area_Condition_Tropical and Subtropical Moist Broadleaf Forests;Tropical and Subtropical Dry Broadleaf Forests,"F8_Area_Condition_Tropical and subtropical grasslands, savannas, and shrublands",F8_Area_Condition_Tundra,F11_Climate_available,F11_Climate_missing,F11_Climate_preliminary,F136_available,F136_missing,F136_preliminary
0,12272.0,1.0,1.0,1.0,301011.40070,0.270918,57.889697,45187.287610,53889.947660,61039.963870,...,0,0,0,0,1,0,0,1,0,0
1,6526.0,1.0,1.0,1.0,86680.35925,0.303313,21.221158,86.386399,109.933379,129.268833,...,0,0,0,0,1,0,0,1,0,0
2,11320.0,2.0,1.0,1.0,297267.15000,0.359829,86.350782,63083.100680,75232.254990,97333.212900,...,0,0,0,0,1,0,0,1,0,0
3,4566.0,0.0,0.0,1.0,92949.66541,0.274426,7.550097,2205.528121,2213.771912,2076.353124,...,0,0,0,0,1,0,0,1,0,0
4,4866.0,1.0,1.0,1.0,154643.44400,0.475473,0.018488,697.059179,1023.343533,1299.903914,...,0,0,0,0,1,0,0,1,0,0


(2486, 56)

In [57]:
X_test = df_test_onehot
Y_test = test['COVID_TCPM']

In [58]:
le = LabelEncoder()
le.fit(Y_test)
le.classes_

LabelEncoder()

array(['low'], dtype=object)

In [60]:
clf_dt = dt(criterion='gini', splitter='best', max_depth=6, min_samples_split=10, min_samples_leaf=5, max_features='auto')
clf_dt = clf_dt.fit(X_test, Y_test)

In [61]:
clf_dt.score(X_train, Y_train)

1.0